<a href="https://colab.research.google.com/github/Amit-Padye/Scraping/blob/main/Downloading_and__adding_file_to_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
from datetime import datetime, timedelta
import psycopg2
import os
import pandas as pd

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36","Accept-Encoding": "identity"}

# Connection parameters
param_dic = {
    "host"      : "203.112.158.89",
    "database"  : "ratings",
    "user"      : "analytics",
    "password"  : "analytics"
}


def connect(params_dic):
    #Making Connection to postgreSQL Server
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
        print("Connection successful")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    return conn


def copy_from_file(conn,table_name,path_of_data):

    f = open(path_of_data, 'r')
    next(f)
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table_name,sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()



def download_and_add_to_database(reportid,folder,fileinitial,filedateformat,part1,table_name,param_dic):
    # check file present or not
    try:
        currdate = datetime.today()
        year = currdate.strftime('%Y')
        month = currdate.strftime('%b').upper()
        todays_date = currdate.strftime(filedateformat)

        filename=fileinitial + todays_date + ".csv"
        if reportid==1:
            url = part1 + year + '/' + month + '/' + filename
        else:
            url = part1 + filename

        if not os.path.isfile(folder+filename):
            response = requests.get(url,verify=False,headers=headers)
            if response.reason =="OK":
                with open(folder + filename, 'wb') as f:
                    f.write(response.content)
                    print('File downloaded')
                    conn = connect(param_dic)
                    path_of_data = folder + filename
                    copy_from_file(conn,table_name,path_of_data)
                    conn.close()

            else:
                for day in range(1,3):
                    currdate = currdate - timedelta(days=day)
                    year = currdate.strftime('%Y')
                    month = currdate.strftime('%b').upper()
                    todays_date = currdate.strftime(filedateformat)
                    filename = fileinitial + todays_date + ".csv"
                    if not os.path.isfile(folder + filename):
                        if reportid == 1:
                            url = part1 + year + '/' + month + '/' + filename
                        else:
                            url = part1 + filename

                        response = requests.get(url,verify=False,headers=headers)
                        if response.reason == "OK":
                            with open(folder + filename, 'wb') as f:
                                f.write(response.content)
                                print('File downloaded')
                                conn = connect(param_dic)
                                path_of_data = folder + filename
                                copy_from_file(conn, table_name, path_of_data)
                                conn.close()
                                break
                    else:
                        break

    except (Exception) as e:
        print("New file not available")


folder="C:/webscraping/"

part1 = 'https://archives.nseindia.com/content/historical/WDM/'
fileinitial="wdmlist_"
filedateformat="%d%m%Y"
table_name=""
download_and_add_to_database(1,folder,fileinitial,filedateformat,part1,table_name,param_dic)

part1 = 'https://archives.nseindia.com/archives/debt/cbm/'
fileinitial="cbm_trd"
filedateformat="%Y%m%d"
table_name=""
download_and_add_to_database(2,folder,fileinitial,filedateformat,part1,table_name,param_dic)

part1 = 'https://archives.nseindia.com/archives/debt/cbm/'
fileinitial="GSEC_SettlementOrderReport_"
filedateformat="%d%m%Y"
table_name=""
download_and_add_to_database(3,folder,fileinitial,filedateformat,part1,table_name,param_dic)




